### 1) 가짜 학생 점수 엑셀 생성

**문제**

Faker를 이용해 학생 이름 10명을 생성하고,NumPy로 국어·영어·수학 점수(0~100)를 만들어 엑셀 파일 `students.xlsx`에 저장하세요.

**조건**

- 시트명: `Scores`
- 컬럼: 이름, 국어, 영어, 수학

**힌트**

- Faker: `fake.name()`
- NumPy: `np.random.randint(0, 101, size=(10, 3))`
- openpyxl: `ws.append()`

In [ ]:
import numpy as np
from pathlib import Path
from faker import Faker
from openpyxl import Workbook

np.random.seed(7)
N = 10
faker = Faker("ko_KR")
faker.seed_instance(7)
# print(faker.name())
# print(faker.email())
# print(faker.job())
names = [faker.name() for _ in range(N)]
emails = [faker.email() for _ in range(N)]
scores = np.random.randint(50, 101, size=(N, 3))  # 50~100점 사이의 정수 난수 생성  
print(names)

total_scores = scores.sum(axis=1) # 각 학생의 총점 계산
avg_scores = scores.mean(axis=1)  # 각 학생의 평균 계산
#print(total_scores)
#print(avg_scores)
wb = Workbook()
ws = wb.active
ws.title = "학생성적"
headers = ["이름", "이메일", "국어", "영어", "수학", "총점", "평균"]
ws.append(headers)
for idx in range(N):
    row = [
        names[idx],
        emails[idx],
        scores[idx, 0],
        scores[idx, 1],
        scores[idx, 2],
        total_scores[idx],
        round(avg_scores[idx], 2)
    ]
    ws.append(row)  
#file_path = Path(__file__).parent / "student_scores.xlsx"    #이건 일반 .py에서는 이렇게 쓰면 된다.
file_path = Path.cwd() / "student_scores.xlsx"  #이건 Jupyter Notebook에서는 이렇게 써야 한다.b.save(file_path)
wb.save(file_path)

['김재호', '박성호', '박광수', '김동현', '류민석', '강진호', '이현지', '고성민', '김성수', '김은경']


### 3) 평균 70점 이상 학생만 필터링

**문제**

학생 평균이 **70점 이상인 학생만** 새로운 시트 `Pass`에 저장하세요.

**힌트**

- 불리언 마스크: `avg >= 70`
- 배열 인덱싱: `data[mask]`

In [ ]:
import numpy as np
from pathlib import Path
from faker import Faker
from openpyxl import Workbook, load_workbook

current_dir = Path(__file__).resolve().parent
wb = load_workbook(current_dir / "student_scores.xlsx")
ws =  wb.active
rows = []

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True):
    rows.append(list(row))

# for row in range(2, ws.max_row + 1):
#     rows.append([
#             ws[f"A{row}"].value,
#             ws[f"B{row}"].value,  
#             ws[f"C{row}"].value,
#             ws[f"D{row}"].value,
#             ws[f"E{row}"].value,
#             ws[f"F{row}"].value,
#             ws[f"G{row}"].value
#     ])
#print(rows)    
data = np.array(rows, dtype=object)  # object로 해야 숫자도 같이 들어감
print(data)
avg_scores = data[:, 6].astype(int)  # 국어, 영어, 수학 점수 부분만 추출하여 정수형으로 변환
print(avg_scores)
mask =  avg_scores >= 70
passed_students = data[mask]
print(passed_students)
wb = Workbook()
ws = wb.active
ws.title = "성적70점이상"
headers = ["이름", "이메일", "국어", "영어", "수학", "총점", "평균"]
ws.append(headers)
for row in passed_students:
    ws.append(row.tolist())
file_path = Path.cwd() / "student_passed.xlsx"    
wb.save(file_path)
    

### 4) 과목별 평균/최대/최소 점수 계산

**문제**

국어·영어·수학 각각에 대해 평균, 최대, 최소 점수를 계산하여 엑셀에 기록하세요.

**힌트**

- `axis=0` → 과목 기준
- `np.mean`, `np.max`, `np.min`

In [ ]:
import numpy as np
from pathlib import Path
from faker import Faker
from openpyxl import Workbook, load_workbook

current_dir = Path(__file__).resolve().parent
wb = load_workbook(current_dir / "student_scores.xlsx")
ws =  wb.active
scores=[]
# 엑셀의 상단 header의 A/B/C...열을 기준으로 값을 읽을 때는 문자열로 접근
for row in range(2, ws.max_row+1):
    scores.append([
                   ws.cell(row,1).value, 
                   ws.cell(row,2).value, 
                   ws.cell(row,3).value, 
                   ws.cell(row,4).value, 
                   ws.cell(row,5).value, 
                   ws.cell(row,6).value, 
                   ws.cell(row,7).value])
print(scores)
data = np.array(scores, dtype=object)
subjects = data[:, 2:5].astype(int)
print(subjects)
subjects_mean = subjects.mean(axis=0)
subjects_max = subjects.max(axis=0)
subjects_min = subjects.min(axis=0)
print(f"과목별 평균 : {subjects_mean}")
print(f"과목별 최대값 : {subjects_max}")    
print(f"과목별 최소값 : {subjects_min}")